# Test arXiv PDF Parser Integration

This notebook tests the arXiv client, PDF parsing via Docling, metadata extraction and chunking workflow.

In [2]:
import os
os.chdir("./..")
print("CWD:", os.getcwd())

CWD: d:\Projects\arxiv-ai-explorer\backend


In [3]:
import asyncio
from pathlib import Path

from src.services.arxiv.client import ArxivClient
from src.services.pdf_parser.factory import make_pdf_parser_service
from src.services.arxiv.metadata_extractor import MetadataExtractor
from src.config import get_settings

c:\Users\amine\anaconda3\envs\arxiv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test 1: Search for recent papers

In [ ]:
settings = get_settings()
client = ArxivClient()
category = (settings.arxiv_categories[0] if settings.arxiv_categories else 'cs.AI')
print(f'Using category: {category}')

papers = await client.search_papers(
    query=f'cat:{category}',
    max_results=3,
    sort_by='submittedDate',
    sort_order='descending',
)
print(f'Found {len(papers)} papers')
for i, paper in enumerate(papers):
    print(f"\n{i+1}. {paper['title'][:80]}...")
    print(f"   arXiv ID: {paper['arxiv_id']}")
    print(f"   PDF URL: {paper['pdf_url']}")

test_paper = papers[0] if papers else None
assert test_paper, 'No papers found for the selected category'

Using category: cs.AI
[ 2025-09-20 12:03:35,806 ] [researchmind] | Module: client |Function: _make_request | Line: 50 - INFO - Fetching arXiv data: http://export.arxiv.org/api/query?search_query=cat%3Acs.AI&start=0&max_results=3&sortBy=submittedDate&sortOrder=descending
[ 2025-09-20 12:03:36,076 ] [researchmind] | Module: client |Function: search_papers | Line: 178 - INFO - Found 3 papers for query: cat:cs.AI
Found 3 papers

1. Generalizable Geometric Image Caption Synthesis...
   arXiv ID: 2509.15217v1
   PDF URL: http://arxiv.org/pdf/2509.15217v1

2. Explicit Context-Driven Neural Acoustic Modeling for High-Fidelity RIR Generatio...
   arXiv ID: 2509.15210v1
   PDF URL: http://arxiv.org/pdf/2509.15210v1

3. FlowRL: Matching Reward Distributions for LLM Reasoning...
   arXiv ID: 2509.15207v1
   PDF URL: http://arxiv.org/pdf/2509.15207v1


In [5]:
test_paper

{'arxiv_id': '2509.15217v1',
 'title': 'Generalizable Geometric Image Caption Synthesis',
 'abstract': 'Multimodal large language models have various practical applications that demand strong reasoning abilities. Despite recent advancements, these models still struggle to solve complex geometric problems. A key challenge stems from the lack of high-quality image-text pair datasets for understanding geometric images. Furthermore, most template-based data synthesis pipelines typically fail to generalize to questions beyond their predefined templates. In this paper, we bridge this gap by introducing a complementary process of Reinforcement Learning with Verifiable Rewards (RLVR) into the data generation pipeline. By adopting RLVR to refine captions for geometric images synthesized from 50 basic geometric relations and using reward signals derived from mathematical problem-solving tasks, our pipeline successfully captures the key features of geometry problem-solving. This enables better ta

## Test 2: Download PDF from arXiv URL

In [6]:
print(f"Testing with paper: {test_paper['title']}")
print(f"arXiv ID: {test_paper['arxiv_id']}")

download_dir = Path('./data/test_downloads')
download_dir.mkdir(parents=True, exist_ok=True)

pdf_path = await client.download_pdf(
    pdf_url="https://arxiv.org/pdf/2509.15207",
    download_path=download_dir / f"{test_paper['arxiv_id'].replace('/', '_')}.pdf",
    max_file_size_mb=150
)
print(f'Downloaded PDF to: {pdf_path}')
print(f'File size: {pdf_path.stat().st_size / (1024*1024):.2f} MB')
print(f'File exists: {pdf_path.exists()}')

Testing with paper: Generalizable Geometric Image Caption Synthesis
arXiv ID: 2509.15217v1
[ 2025-09-20 12:03:36,142 ] [researchmind] | Module: client |Function: download_pdf | Line: 272 - INFO - Downloading PDF from: https://arxiv.org/pdf/2509.15207
[ 2025-09-20 12:03:36,385 ] [researchmind] | Module: client |Function: download_pdf | Line: 296 - INFO - Downloaded PDF (0.9MB) to: data\test_downloads\2509.15217v1.pdf
Downloaded PDF to: data\test_downloads\2509.15217v1.pdf
File size: 0.88 MB
File exists: True


## Test 3: Parse downloaded PDF with Docling

In [7]:
pdf_parser = make_pdf_parser_service()
parsed_content = await pdf_parser.parse_pdf(pdf_path)

    

[ 2025-09-20 12:03:36,433 ] [docling.datamodel.document] | Module: document |Function: _guess_format | Line: 328 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
[ 2025-09-20 12:03:36,628 ] [docling.document_converter] | Module: document_converter |Function: _convert | Line: 318 - INFO - Going to convert document batch...
[ 2025-09-20 12:03:36,629 ] [docling.document_converter] | Module: document_converter |Function: _get_pipeline | Line: 363 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 60c8066c482b9239b869b997da3fb1da
[ 2025-09-20 12:03:37,806 ] [docling.models.factories.base_factory] | Module: base_factory |Function: load_from_plugins | Line: 112 - INFO - Loading plugin 'docling_defaults'
[ 2025-09-20 12:03:37,811 ] [docling.models.factories.base_factory] | Module: base_factory |Function: load_from_plugins | Line: 104 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
[ 2025

In [8]:
parsed_content

PdfContent(sections=[PaperSection(title='Content', content='arXiv:2509.15207v1  [cs.LG]  18 Sep 2025\nFlowRL: Matching Reward Distributions for LLM Reasoning\n2025-09-17', level=1), PaperSection(title='FlowRL: Matching Reward Distributions for LLM Reasoning', content='Xuekai Zhu 1 , Daixuan Cheng 6 , Dinghuai Zhang 3 , Hengli Li 5 , Kaiyan Zhang 4 , Che Jiang 4 , Youbang Sun 4 , Ermo Hua 4 , Yuxin Zuo 4 , Xingtai Lv 4 , Qizheng Zhang 7 , Lin Chen 1 , Fanghao Shao 1 , Bo Xue 1 , Yunchong Song 1 , Zhenjie Yang 1 , Ganqu Cui 2 , Ning Ding 4 , 2 , Jianfeng Gao 3 , Xiaodong Liu 3 , Bowen Zhou 4 , 2 ‡ , Hongyuan Mei 8 ‡ , Zhouhan Lin 1 , 2 ‡\n1 Shanghai Jiao Tong University 2 Shanghai AI Laboratory 3 Microsoft Research 4 Tsinghua University 5 Peking University 6 Renmin University of China 7 Stanford University 8 Toyota Technological Institute at Chicago\n/envelope hongyuanmei@gmail.com /envelope xuekaizhu0@gmail.com /github FlowRL ‡ Corresponding Authors.\nAbstract | We propose FlowRL: match

## Test 4: Build paper_with_content and extract metadata

In [9]:
from src.services.arxiv.metadata_extractor import MetadataExtractor


In [10]:
paper_with_content = {**test_paper}
paper_with_content['content'] = parsed_content.raw_text if parsed_content else ''
paper_with_content['sections'] = [
    {'title': s.title, 'content': s.content} for s in (parsed_content.sections if parsed_content else [])
]
paper_with_content['is_processed'] = True

metadata_extractor = MetadataExtractor()
enriched = await metadata_extractor.extract_metadata(paper_with_content)



[ 2025-09-20 12:04:52,116 ] [researchmind] | Module: metadata_extractor |Function: extract_metadata | Line: 177 - INFO - Extracting metadata for paper: 2509.15217v1
[ 2025-09-20 12:04:52,185 ] [researchmind] | Module: metadata_extractor |Function: extract_metadata | Line: 251 - INFO - Extracted metadata: 1 metrics


In [11]:
paper_with_content.update(enriched)


In [12]:
paper_with_content

{'arxiv_id': '2509.15217v1',
 'title': 'Generalizable Geometric Image Caption Synthesis',
 'abstract': 'Multimodal large language models have various practical applications that demand strong reasoning abilities. Despite recent advancements, these models still struggle to solve complex geometric problems. A key challenge stems from the lack of high-quality image-text pair datasets for understanding geometric images. Furthermore, most template-based data synthesis pipelines typically fail to generalize to questions beyond their predefined templates. In this paper, we bridge this gap by introducing a complementary process of Reinforcement Learning with Verifiable Rewards (RLVR) into the data generation pipeline. By adopting RLVR to refine captions for geometric images synthesized from 50 basic geometric relations and using reward signals derived from mathematical problem-solving tasks, our pipeline successfully captures the key features of geometry problem-solving. This enables better ta

In [13]:
paper_with_content

{'arxiv_id': '2509.15217v1',
 'title': 'Generalizable Geometric Image Caption Synthesis',
 'abstract': 'Multimodal large language models have various practical applications that demand strong reasoning abilities. Despite recent advancements, these models still struggle to solve complex geometric problems. A key challenge stems from the lack of high-quality image-text pair datasets for understanding geometric images. Furthermore, most template-based data synthesis pipelines typically fail to generalize to questions beyond their predefined templates. In this paper, we bridge this gap by introducing a complementary process of Reinforcement Learning with Verifiable Rewards (RLVR) into the data generation pipeline. By adopting RLVR to refine captions for geometric images synthesized from 50 basic geometric relations and using reward signals derived from mathematical problem-solving tasks, our pipeline successfully captures the key features of geometry problem-solving. This enables better ta

# Test 5 : Chunker test 


In [14]:
from src.services.chunking.chunker import PaperChunker

In [15]:
paper_chunker = PaperChunker()

chunks = paper_chunker.chunk_paper(enriched)

d:\Projects\arxiv-ai-explorer\backend\src\services\chunking\chunker.py:47: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=self.config.embedding_model)


[ 2025-09-20 12:04:53,005 ] [sentence_transformers.SentenceTransformer] | Module: SentenceTransformer |Function: __init__ | Line: 219 - INFO - Use pytorch device_name: cpu
[ 2025-09-20 12:04:53,005 ] [sentence_transformers.SentenceTransformer] | Module: SentenceTransformer |Function: __init__ | Line: 227 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [16]:
chunks

[{'arxiv_id': '2509.15217v1',
  'title': 'Generalizable Geometric Image Caption Synthesis',
  'primary_category': 'cs.AI',
  'categories': ['cs.AI', 'cs.CV', 'cs.LG'],
  'section_title': 'Content',
  'section_type': 'Content',
  'chunk_index': 0,
  'total_chunks': 1,
  'chunk_text': 'arXiv:2509.15207v1  [cs.LG]  18 Sep 2025\nFlowRL: Matching Reward Distributions for LLM Reasoning\n2025-09-17',
  'start_char': 0,
  'end_char': 107,
  'published_date': '2025-09-18T17:59:11Z',
  'authors': ['Yue Xin',
   'Wenyuan Wang',
   'Rui Pan',
   'Ruida Wang',
   'Howard Meng',
   'Renjie Pi',
   'Shizhe Diao',
   'Tong Zhang'],
  'word_count': 13},
 {'arxiv_id': '2509.15217v1',
  'title': 'Generalizable Geometric Image Caption Synthesis',
  'primary_category': 'cs.AI',
  'categories': ['cs.AI', 'cs.CV', 'cs.LG'],
  'section_title': 'FlowRL: Matching Reward Distributions for LLM Reasoning',
  'section_type': 'FlowRL: Matching Reward Distributions for LLM Reasoning',
  'chunk_index': 0,
  'total_chu

In [17]:
enriched.keys()

dict_keys(['arxiv_id', 'title', 'abstract', 'authors', 'categories', 'primary_category', 'published', 'updated', 'pdf_url', 'arxiv_url', 'doi', 'journal_ref', 'content', 'sections', 'is_processed', 'metrics', 'research_area', 'research_areas_all', 'word_count', 'author_count', 'institutions'])

dict_keys(['arxiv_id', 'title', 'summary', 'authors', 'categories', 'primary_category', 'published', 'updated', 'pdf_url', 'abs_url', 'doi', 'journal_ref', 'content', 'sections', 'is_processed', 'datasets', 'metrics', 'research_area', 'research_areas_all', 'word_count', 'author_count', 'institutions'])


In [18]:
enriched

{'arxiv_id': '2509.15217v1',
 'title': 'Generalizable Geometric Image Caption Synthesis',
 'abstract': 'Multimodal large language models have various practical applications that demand strong reasoning abilities. Despite recent advancements, these models still struggle to solve complex geometric problems. A key challenge stems from the lack of high-quality image-text pair datasets for understanding geometric images. Furthermore, most template-based data synthesis pipelines typically fail to generalize to questions beyond their predefined templates. In this paper, we bridge this gap by introducing a complementary process of Reinforcement Learning with Verifiable Rewards (RLVR) into the data generation pipeline. By adopting RLVR to refine captions for geometric images synthesized from 50 basic geometric relations and using reward signals derived from mathematical problem-solving tasks, our pipeline successfully captures the key features of geometry problem-solving. This enables better ta

## Cleanup

In [ ]:
# import shutil
# if download_dir.exists():
#     shutil.rmtree(download_dir)
# print('Test completed!')

Test completed!
